<a href="https://colab.research.google.com/github/minnieteng/ivado-mila-dl-school-2021/blob/main/Assignment_3_Computer_Vision_Melanoma_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Welcome to this Michener Institute & Vector Institute Course!

This is a case study in the ‘AI for Clinician Champions Certificate' program! 

https://michener.ca/ce_course/ai-for-clinician-champions-certificate-program/ - 
This program is offered by Michener Institute & Vector Institute for clinicians who wish to learn more about AI in Healthcare. 
This coding tutorial is optional however, themes from the model will be used in the overall assignment. Learners are enouraged to, at a minimum, look through this tutorial or, run the model for a full learning experience. 

This week's case study will introduce you to medical image analysis, using melanoma classification as an example. The goal of this case study is to build neural network models to classify whether there is melanoma in skin lesion images. This is a Computer Vision (CV) use-case of AI. CV in AI deals with how models deal with images or video.

Instructor: Dr. Devin Singh (@DrDevSK) | Assignment Developer: Jianan Chen | Course Tutors: Jianan Chen, Flora Wan, and S. Alex Yun | Course Director: Shingai Manjengwa (@Tjido) 
 
***Never stop learning!***

# Melanoma classification

Skin cancer is the most common cancer type. Melanoma, specifically, accounts for 75% of skin cancer deaths, despite being the least common skin cancer. It is estimated by the American Cancer Society that there will be over 100,000 new melanoma cases and more than 7,000 melanoma-related deaths each year. Early detection of melanoma enabled by machine learning may improve patient prognosis. 

In this case study, we build deep learning models to identify melanomas in images of skin lesions based on data from the SIIM-ISIC Melanoma Classification Challenge. The SSIM-ISIC Melanoma Classfication Challenge is a [Kaggle competition](https://www.kaggle.com/c/siim-isic-melanoma-classification) hosted by [Society for Imaging Informatics in Medicine](https://siim.org/) and [the International Skin Imaging Collaboration](https://www.isic-archive.com/#!/topWithHeader/wideContentTop/main). We will cover several aspects of computer vision with deep learning, including network structures, interpretation of neural networks and dataset constructions.

Please read the instructions before running the code. For each block, the instruction will be on the right column.  Please try to avoid refreshing the webpage or restarting the session. If you refresh, please run every block again from the beginning.

**Acknowledgements**: The code in this case study is adpated from [Andrada Olteanu](https://www.kaggle.com/andradaolteanu/melanoma-competiton-aug-resnet-effnet-lb-0-91) and [Roman's](https://www.kaggle.com/nroman/melanoma-pytorch-starter-efficientnet/output) work, and the format of the notebook is inspired by the explanatory notebook in [MIDOG challenge](https://imi.thi.de/midog/). 

In [ ]:
#@title Enabling GPU in Google Colab (**Important**)

#@markdown - Before we proceed, we need to turn on GPU acceleration for this notebook, this can be done by
#@markdown selecting the following menu options: Runtime / Change runtime type / Hardware accelerator.
#@markdown Select 'GPU' from the dropdown menu and press 'Save'.
#@markdown - Once you have changed the settings, run this block and check the output. If it says "Device available now: cuda" then we are good to go!
#@markdown - GPU stands for Graphical Processing Unit (compare to CPU, Central Processing Unit). GPUs allow for faster and more efficient processing of images in computing systems.

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

In [ ]:
#@title Import some python packages { vertical-output: true, display-mode: "form" }
#@markdown - In this case study we focus on images taken by cameras (.jpg, .png, ..) so we import [opencv](https://docs.opencv.org/master/d6/d00/tutorial_py_root.html)
#@markdown - For other image modalities e.g. MR, CT and US, we can use specialized packages such as [SimpleITK](https://simpleitk.org/) and [pydicom](https://pydicom.github.io/).
#@markdown 

%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install -U efficientnet_pytorch

import json
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import random
import cv2

In [ ]:
#@title Download data to Colab { run: "auto", vertical-output: true, display-mode: "both" }


from IPython.display import clear_output
import zipfile

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1P62dCd6Nu8pkV9YPMb3QCfMBOZUVAT2Y' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1P62dCd6Nu8pkV9YPMb3QCfMBOZUVAT2Y" -O 'casestudy2.zip' && rm -rf /tmp/cookies.txt

with zipfile.ZipFile('casestudy2.zip', 'r') as zip_ref:
    zip_ref.extractall('casestudy2')

#@markdown The dataset is stored on Google drive as a zip file. When we run the cell
#@markdown Colab will retrieve this zip file and unzip it into images and spreadsheets.

#@markdown You can have a look at the dataset structure by pressing the folder icon on the left sidebar.

SIIM_folder = Path("/content/casestudy2/Case 3") 

#@markdown Your output should contain **test_clean.csv** and **train_clean.csv**:
print(list(SIIM_folder.glob("*.*")))

In [ ]:
#@title Library for visualization and deep learning { vertical-output: true, display-mode: "form" }

#@markdown In this case study we will be using [PyTorch](https://pytorch.org/) for building and training the deep neural network.

#@markdown PyTorch is one of the most popular open-source deep learning framework for researchers.
import pandas as pd
import torchvision.models as models
from torch import nn
from torchvision import datasets, transforms
import random
import numpy as np
from sklearn.model_selection import GroupKFold, train_test_split
import gc
import time
import datetime
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, roc_auc_score
import os
import PIL
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

print('Package imported!')

In [ ]:
#@title Let us take a look at the files { display-mode: "form" }

#@markdown The original SIIM-ISIC dataset contains more than 44,000 images. In 
#@markdown order to speed up training, we randomly selected 600 images 
#@markdown (300 benign and 300 malignant skin lesions) for our case study and 
#@markdown split the dataset into 80% for training and 20% for testing.

image_folder = SIIM_folder / "SIIM"
train_csv = SIIM_folder / "train_clean.csv"
test_csv = SIIM_folder / "test_clean.csv"
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
example_df = train_df.loc[:, ['image_name','benign_malignant','target']]

print(f'There are {len(train_df)} images for training and {len(test_df)} images for testing.')
print('First five rows of our dataframe:')
print(example_df.head())

fig = plt.figure(figsize=(8, 8))
columns = 5
rows = 1
ax = []

for i, img in enumerate(train_df.image_name[0:5]):
    img_path = os.path.join('/content/casestudy2/Case 3/SIIM/', img + '.jpg')
    imgs = cv2.imread(img_path)
    imgs = cv2.cvtColor(imgs, cv2.COLOR_BGR2RGB)
    imgs = np.array(imgs)
    ax.append(fig.add_subplot(rows, columns, i+1))
    ax[-1].set_title(img)  # set title
    plt.imshow(imgs)
    plt.axis("off")
plt.show()


In [ ]:
#@title Set seed to ensure the results are reproducible { display-mode: "form" }

#@markdown It's important to make our results reproducible so that it's easier to debug your model and fairly evaluate model performance.
#@markdown This can be achived by setting a random seed for all the packages we have using.
def set_seed(seed=1234):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

print('The defualt seed for this notebook is 1234.')

# Neural Network
![alt text](https://drive.google.com/uc?export=view&id=1fCSKpKfmW-mrAdAW-rL40lixy3thHaGw)
**Image credit**: [Google AI blog](https://ai.googleblog.com/2019/05/efficientnet-improving-accuracy-and.html)

Various neural network architectures are proposed in the recent years. The above
figure shows the trade-off between the number of parameters (demand of computing power & training time) and performance of a few popular network structures. In this study we will implement EfficientNet-B2 and ResNet-50 for comparision.

In [ ]:
#@title Create dataset and dataloader { display-mode: "form" }

#@markdown - To train neural networks on your own data, you need to first create
#@markdown custom datasets and dataloaders for your data.
#@markdown - We create a SIIM melanoma dataset and apply augmentations to our data
#@markdown to enhance our dataset.
#@markdown - Data augmentation is a technique that is used in almost all computer vision projects, 
#@markdown for generating more data and taking into account the variations in unseen test data.
#@markdown - You will have a chance to tune the data augmentations parameters in the following cells.
#@markdown - A review on data augmentation if you are interested: [link to paper](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0197-0)

import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.transforms import *
import cv2
import os
import numpy as np

class MelanomaDataset(Dataset):
    def __init__(self, dataframe, vertical_flip, horizontal_flip, color_jitter=0.3,
                 is_train=True, is_valid=False, is_test=False, is_original=False):
        self.dataframe, self.is_train, self.is_valid, self.is_original = dataframe, is_train, is_valid, is_original
        self.vertical_flip, self.horizontal_flip = vertical_flip, horizontal_flip
        self.color_jitter=color_jitter

        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        # Data Augmentation (custom for each dataset type)
        if is_train or is_test:
            self.transform = Compose([ToTensor(),
                                      RandomResizedCrop((224,224), (0.6, 1.0)),
                                      RandomAffine(90, scale=(0.8, 1.2)),
                                      RandomHorizontalFlip(p=self.horizontal_flip),
                                      RandomVerticalFlip(p=self.vertical_flip),
                                      ColorJitter(brightness=self.color_jitter, contrast=self.color_jitter, saturation=self.color_jitter, hue=0.1),
                                      normalize])
        elif is_valid:
            self.transform = Compose([ToTensor(),
                                      Resize((224,224)),
                                      normalize])
        else:
            self.transform = Compose([ToTensor(),
                                      Resize((224,224))])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        # Select path and read image
        image_name = self.dataframe['image_name'][index]
        image_path = os.path.join('/content/casestudy2/Case 3/SIIM/', image_name + '.jpg') 
        image = cv2.imread(image_path)
        # For this image also import .csv information (sex, age, anatomy)
        csv_data = np.array(self.dataframe.iloc[index][['sex', 'age', 'anatomy']].values,
                            dtype=np.float32)

        # Apply transforms
        image = self.transform(image)
        # Extract image from dictionary

        # If train/valid: image + class | If test: only image
        if self.is_train or self.is_valid or self.is_original:
            return (image, csv_data), self.dataframe['target'][index]
        else:
            return (image, csv_data)



In [ ]:
#@title Define network structures
#@markdown In this block we define the network structures.
from torch import nn
import torch
from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet50
import torch.nn.functional as F

class EfficientNetwork(nn.Module):
    def __init__(self, output_size, b4=False, b2=False):
        super().__init__()
        self.b4, self.b2 = b4, b2

        # Define Feature part (IMAGE)
        if b4:
            self.features = EfficientNet.from_pretrained('efficientnet-b4')
        elif b2:
            self.features = EfficientNet.from_pretrained('efficientnet-b2')
        else:
            self.features = EfficientNet.from_pretrained('efficientnet-b7')

        # Define Classification part
        if b4:
            self.classification = nn.Sequential(nn.Linear(1792, output_size))
        elif b2:
            self.classification = nn.Sequential(nn.Linear(1408, output_size))
        else:
            self.classification = nn.Sequential(nn.Linear(2560, output_size))

    def forward(self, image, prints=False):

        # IMAGE CNN
        image = self.features.extract_features(image)

        if self.b4:
            image = F.avg_pool2d(image, image.size()[2:]).reshape(-1, 1792)
        elif self.b2:
            image = F.avg_pool2d(image, image.size()[2:]).reshape(-1, 1408)
        else:
            image = F.avg_pool2d(image, image.size()[2:]).reshape(-1, 2560)

        # CLASSIF
        out = self.classification(image)

        return out

class ResNet50Network(nn.Module):
    def __init__(self, output_size, no_columns):
        super().__init__()
        self.no_columns, self.output_size = no_columns, output_size

        # Define Feature part (IMAGE)
        self.features = resnet50(pretrained=True)  # 1000 neurons out

        # Define Classification part
        self.classification = nn.Linear(1000, output_size)

    def forward(self, image, prints=False):

        # Image CNN
        image = self.features(image)

        # CLASSIF
        out = self.classification(image)

        return out

In [ ]:
#@title Define training function
#@markdown In this block we implemete a function for training, validating and testing the network.
def train_folds(preds_submission, model, train_df, test_df, flip=0.5, color_jitter=0.3, version='v1'):
    # Creates a .txt file that will contain the logs
    f = open(f"logs_{version}.txt", "w+")

    # ----- STATICS -----
    indices = np.arange(len(train_df))
    train_index, valid_index = train_test_split(indices, test_size=0.2, random_state=1234)
    train_data = train_df.iloc[train_index].reset_index(drop=True)
    valid_data = train_df.iloc[valid_index].reset_index(drop=True)

    # Append to .txt
    with open(f"logs_{version}.txt", 'a+') as f:
        print('-' * 10, 'Training', '-' * 10, file=f)
    print('-' * 10, 'Training', '-' * 10)

    # --- Create Instances ---
    # Best ROC score in this fold
    best_roc = None
    # Reset patience before every fold
    patience_f = patience

    # Initiate the model
    model = model

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer=optimizer, mode='max',
                                  patience=lr_patience, verbose=True, factor=lr_factor)
    criterion = nn.BCEWithLogitsLoss()

    # Create Data instances
    train = MelanomaDataset(train_data, vertical_flip=flip, horizontal_flip=flip, color_jitter=color_jitter,
                            is_train=True, is_valid=False, is_test=False)
    valid = MelanomaDataset(valid_data, vertical_flip=flip, horizontal_flip=flip, color_jitter=color_jitter,
                            is_train=False, is_valid=True, is_test=False)
    # Read in test data | Remember! We're using data augmentation like we use for Train data.
    test = MelanomaDataset(test_df, vertical_flip=flip, horizontal_flip=flip, color_jitter=color_jitter,
                           is_train=False, is_valid=False, is_test=True)

    # Dataloaders
    train_loader = DataLoader(train, batch_size=batch_size1, shuffle=True, num_workers=num_workers)
    # shuffle=False! Otherwise function won't work!!!
    # how do I know? ^^
    valid_loader = DataLoader(valid, batch_size=batch_size2, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test, batch_size=batch_size2, shuffle=False, num_workers=num_workers)

    # === EPOCHS ===
    for epoch in range(epochs):
        start_time = time.time()
        correct = 0
        train_losses = 0

        # === TRAIN ===
        # Sets the module in training mode.
        model.train()

        for (images, csv_data), labels in train_loader:
            # Save them to device
            images = images.type(torch.float32).cuda()
            csv_data = csv_data.type(torch.float32).cuda()
            labels = labels.type(torch.float32).cuda()

            # Clear gradients first; very important, usually done BEFORE prediction
            optimizer.zero_grad()

            # Log Probabilities & Backpropagation
            out = model(images, csv_data)
            loss = criterion(out, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()

            # --- Save information after this batch ---
            # Save loss
            train_losses += loss.item()
            # From log probabilities to actual probabilities
            train_preds = torch.round(torch.sigmoid(out))  # 0 and 1
            # Number of correct predictions
            correct += (train_preds.cpu() == labels.cpu().unsqueeze(1)).sum().item()

        # Compute Train Accuracy
        train_acc = correct / len(train_index)

        # === EVAL ===
        # Sets the model in evaluation mode
        model.eval()

        # Create matrix to store evaluation predictions (for accuracy)
        valid_preds = torch.zeros(size=(len(valid_index), 1), device=device, dtype=torch.float32)

        # Disables gradients (we need to be sure no optimization happens)
        with torch.no_grad():
            for k, ((images, csv_data), labels) in enumerate(valid_loader):
                images = images.type(torch.float32).cuda()
                csv_data = csv_data.type(torch.float32).cuda()
                labels = labels.type(torch.float32).cuda()
                # images = torch.tensor(images, device=device, dtype=torch.float32)
                # csv_data = torch.tensor(csv_data, device=device, dtype=torch.float32)
                # labels = torch.tensor(labels, device=device, dtype=torch.float32)

                out = model(images, csv_data)
                pred = torch.sigmoid(out)
                valid_preds[k * images.shape[0]: k * images.shape[0] + images.shape[0]] = pred

            # Compute accuracy
            valid_acc = accuracy_score(valid_data['target'].values,
                                       torch.round(valid_preds.cpu()))
            # Compute ROC
            valid_roc = roc_auc_score(valid_data['target'].values,
                                      valid_preds.cpu())

            # Compute time on Train + Eval
            duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]

            # PRINT INFO
            # Append to .txt file
            with open(f"logs_{version}.txt", 'a+') as f:
                print('{} | Epoch: {}/{} | Loss: {:.4} | Train Acc: {:.3} | Valid Acc: {:.3} | ROC: {:.3}'. \
                      format(duration, epoch + 1, epochs, train_losses, train_acc, valid_acc, valid_roc), file=f)
            # Print to console
            print('{} | Epoch: {}/{} | Loss: {:.4} | Train Acc: {:.3} | Valid Acc: {:.3} | ROC: {:.3}'. \
                  format(duration, epoch + 1, epochs, train_losses, train_acc, valid_acc, valid_roc))

            # === SAVE MODEL ===

            # Update scheduler (for learning_rate)
            scheduler.step(valid_roc)

            # # Update best_roc
            # if not best_roc:  # If best_roc = None
            #     best_roc = valid_roc
            #     torch.save(model.state_dict(),
            #                f"Epoch{epoch + 1}_ValidAcc_{valid_acc:.3f}_ROC_{valid_roc:.3f}.pth")
            #     continue

            # if valid_roc > best_roc:
            #     best_roc = valid_roc
            #     # Reset patience (because we have improvement)
            #     patience_f = patience
            #     # torch.save(model.state_dict(),
            #     #            f"Epoch{epoch + 1}_ValidAcc_{valid_acc:.3f}_ROC_{valid_roc:.3f}.pth")
            # else:

            #     # Decrease patience (no improvement in ROC)
            #     patience_f = patience_f - 1
            #     if patience_f == 0:
            #         with open(f"logs_{version}.txt", 'a+') as f:
            #             print('Early stopping (no improvement since 3 models) | Best ROC: {}'. \
            #                   format(best_roc), file=f)
            #         print('Early stopping (no improvement since 3 models) | Best ROC: {}'. \
            #               format(best_roc))
            #         break

    return model

In [ ]:
#@title Define hyperparameters
#@markdown In this block we define the hyperparameters used in training, for example number of epochs, learning rate and batch size. 
#@markdown Choice of hyperparameters can have a big impact on network performance. 

set_seed()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# ----- STATICS, don't change -----
output_size = 1
csv_columns = ['sex', 'age', 'anatomy']
no_columns = 3
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
train_len = len(train_df)
test_len = len(test_df)
# -------------------

# Out of Fold Predictions
oof = np.zeros(shape=(train_len, 1))

# Predictions
preds_submission = torch.zeros(size=(test_len, 1), dtype=torch.float32, device=device)

# ----- Hyperparameters -----
# probability of applying vertical/horinzontal flip
flip = 0.5
color_jitter = 0.3
# number of epochs for training
epochs = 10
# learning rate
learning_rate = 0.0005
# batch_size
batch_size1 = 4
batch_size2 = 4
# ----- -----
patience = 3
num_workers = 0
weight_decay = 0.0
lr_patience = 1  # 1 model not improving until lr is decreasing
lr_factor = 0.4  # by how much the lr is decreasing
# -------------------



In [ ]:
#@title EfficientNet-B2
#@markdown - EfficientNet is a state-of-the-art neural network model. It has both superior performance and lower number of parameters compared with previous models.
#@markdown - The number in the model name reflects the depths/parameters of the model, where a larger number means a larger model. Here we use B2 out of B0-B7.
#@markdown - This training is going to take a while. Depending on the GPU you are getting with colab, it may range from **25 to 50 minutes**.
#@markdown - For the EfficientNet B2 model, you may reach a validation AUC of around 0.85

#@markdown You can tune some hyperparameters using the slide bars below.

#@markdown - **Flip probability** controls the probability for applying horizontal and vertical flipping to the images, where 0 means not flipping.
#@markdown - **Color jitter** controls the strengths of color alterations applied to the images, including brightness, contrast, saturation and hue, where 0 means no color augmentations. 
#@markdown - The default value may not be the optimal setting. In general we will get 
#@markdown better performance with higher values, until the performance drops when color jitter value becomes too large.
flip = 0.5 #@param {type:"slider", min:0, max:0.5, step:0.1}
color_jitter = 0.3 #@param {type:"slider", min:0.0, max:1, step:0.1}
#@markdown **Note:** after you change the hyperparameters you will need to rerun this cell to see the difference in performance. This will take another 25-50 minutes and is optional.
#epochs = 10 #@param {type:"slider", min:3, max:20, step:1}
epochs = 10


train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
# --- EffNet B2 ---
model = EfficientNetwork(output_size=output_size, b4=False, b2=True).to(device)

# # ===== Uncomment and Train =====
trained_model = train_folds(preds_submission = preds_submission, model = model, train_df = train_df, test_df=test_df, flip = flip, color_jitter=color_jitter, version = 'v1')

In [ ]:
#@title ResNet-50 (**optional**)
#@markdown - ResNet is one of the most widely used and successful neural network structures [link to paper](https://arxiv.org/abs/1512.03385).
#@markdown - If you are interested, run this block and compare the performance of ResNet-50 with EfficientNet-B2.
#@markdown - This is likely going to take longer to train compared to EfficientNet-B2. - This training is going to take a while. Depending on the GPU you are getting with colab, it may range from **45 to 75 minutes**.
#@markdown - For the ResNet50 model, you may reach an validation AUC of around 0.80

#@markdown You can change the augmentation parameters using the slide bars below.

#@markdown - **Flip probability** controls the probability for applying horizontal and vertical flipping to the images.
#@markdown - **Color jitter** controls the strengths of color alterations applied to the images, including brightness, contrast, saturation and hue. 
#@markdown - The default value may not be the optimal settings. In general we will get 
#@markdown better performance with higher values, until the performance drops when color jitter value becomes too large.
flip = 0.5 #@param {type:"slider", min:0, max:0.5, step:0.1}
color_jitter = 0.3 #@param {type:"slider", min:0.0, max:1, step:0.1}
#@markdown **Note:** after you change the hyperparameters you will need to rerun this cell to see the difference in performance. This will take another 45-75 minutes and is optional.
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
#--- ResNet50 ---
model = ResNet50Network(output_size=output_size, no_columns=no_columns).to(device)
trained_model = train_folds(preds_submission = preds_submission, model = model, train_df = train_df, test_df=test_df, flip = flip, color_jitter=color_jitter, version = 'v1')

# Interpreting neural networks

Machine learning, especially deep learning algorithms are sometimes referred to as a black box, as we just feed in some data and receive an output, and we usually don't know what happens in the middle. Interpretation of neural networks has been an active field of research and there are a few interesting and powerful techniques that can open the 'black box' for us. For example, [GradCAM](https://arxiv.org/abs/1610.02391) and its variations can highlight the regions that affects the decision of neural networks.

In [ ]:
#@title Define GradCAM, GradCAM++ and utility functions
class GradCAM(object):
    """Calculate GradCAM salinecy map.

    A simple example:

        # initialize a model, model_dict and gradcam
        resnet = torchvision.models.resnet101(pretrained=True)
        resnet.eval()
        model_dict = dict(model_type='resnet', arch=resnet, layer_name='layer4', input_size=(224, 224))
        gradcam = GradCAM(model_dict)

        # get an image and normalize with mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
        img = load_img()
        normed_img = normalizer(img)

        # get a GradCAM saliency map on the class index 10.
        mask, logit = gradcam(normed_img, class_idx=10)

        # make heatmap from mask and synthesize saliency map using heatmap and img
        heatmap, cam_result = visualize_cam(mask, img)


    Args:
        model_dict (dict): a dictionary that contains 'model_type', 'arch', layer_name', 'input_size'(optional) as keys.
        verbose (bool): whether to print output size of the saliency map givien 'layer_name' and 'input_size' in model_dict.
    """

    def __init__(self, model_dict, verbose=False):
        model_type = model_dict['type']
        layer_name = model_dict['layer_name']
        self.model_arch = model_dict['arch']

        self.gradients = dict()
        self.activations = dict()

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        if 'vgg' in model_type.lower():
            target_layer = find_vgg_layer(self.model_arch, layer_name)
        elif 'resnet' in model_type.lower():
            target_layer = find_resnet_layer(self.model_arch, layer_name)
        elif 'densenet' in model_type.lower():
            target_layer = find_densenet_layer(self.model_arch, layer_name)
        elif 'alexnet' in model_type.lower():
            target_layer = find_alexnet_layer(self.model_arch, layer_name)
        elif 'squeezenet' in model_type.lower():
            target_layer = find_squeezenet_layer(self.model_arch, layer_name)
        elif 'efficientnet' in model_type.lower():
            target_layer = self.model_arch.features._conv_head

        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        if verbose:
            try:
                input_size = model_dict['input_size']
            except KeyError:
                print("please specify size of input image in model_dict. e.g. {'input_size':(224, 224)}")
                pass
            else:
                device = 'cuda' if next(self.model_arch.parameters()).is_cuda else 'cpu'
                self.model_arch(torch.zeros(1, 3, *(input_size), device=device))
                print('saliency_map size :', self.activations['value'].shape[2:])

    def forward(self, input, class_idx=None, retain_graph=False):
        """
        Args:
            input: input image with shape of (1, 3, H, W)
            class_idx (int): class index for calculating GradCAM.
                    If not specified, the class index that makes the highest model prediction score will be used.
        Return:
            mask: saliency map of the same spatial dimension with input
            logit: model output
        """
        b, c, h, w = input.size()

        logit = self.model_arch(input)
        if class_idx is None:
            score = logit[:, logit.max(1)[-1]].squeeze()
        else:
            score = logit[:, class_idx].squeeze()

        self.model_arch.zero_grad()
        score.backward(retain_graph=retain_graph)
        gradients = self.gradients['value']
        activations = self.activations['value']
        b, k, u, v = gradients.size()

        alpha = gradients.view(b, k, -1).mean(2)
        # alpha = F.relu(gradients.view(b, k, -1)).mean(2)
        weights = alpha.view(b, k, 1, 1)

        saliency_map = (weights * activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data

        return saliency_map, logit

    def __call__(self, input, class_idx=None, retain_graph=False):
        return self.forward(input, class_idx, retain_graph)


class GradCAMpp(GradCAM):
    """Calculate GradCAM++ salinecy map.

    A simple example:

        # initialize a model, model_dict and gradcampp
        resnet = torchvision.models.resnet101(pretrained=True)
        resnet.eval()
        model_dict = dict(model_type='resnet', arch=resnet, layer_name='layer4', input_size=(224, 224))
        gradcampp = GradCAMpp(model_dict)

        # get an image and normalize with mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
        img = load_img()
        normed_img = normalizer(img)

        # get a GradCAM saliency map on the class index 10.
        mask, logit = gradcampp(normed_img, class_idx=10)

        # make heatmap from mask and synthesize saliency map using heatmap and img
        heatmap, cam_result = visualize_cam(mask, img)


    Args:
        model_dict (dict): a dictionary that contains 'model_type', 'arch', layer_name', 'input_size'(optional) as keys.
        verbose (bool): whether to print output size of the saliency map givien 'layer_name' and 'input_size' in model_dict.
    """

    def __init__(self, model_dict, verbose=False):
        super(GradCAMpp, self).__init__(model_dict, verbose)

    def forward(self, input, class_idx=None, retain_graph=False):
        """
        Args:
            input: input image with shape of (1, 3, H, W)
            class_idx (int): class index for calculating GradCAM.
                    If not specified, the class index that makes the highest model prediction score will be used.
        Return:
            mask: saliency map of the same spatial dimension with input
            logit: model output
        """
        b, c, h, w = input.size()

        logit = self.model_arch(input)
        if class_idx is None:
            score = logit[:, logit.max(1)[-1]].squeeze()
        else:
            score = logit[:, class_idx].squeeze()

        self.model_arch.zero_grad()
        score.backward(retain_graph=retain_graph)
        gradients = self.gradients['value']  # dS/dA
        activations = self.activations['value']  # A
        b, k, u, v = gradients.size()

        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
                      activations.mul(gradients.pow(3)).view(b, k, u * v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))

        alpha = alpha_num.div(alpha_denom + 1e-7)
        positive_gradients = F.relu(score.exp() * gradients)  # ReLU(dY/dA) == ReLU(exp(S)*dS/dA))
        weights = (alpha * positive_gradients).view(b, k, u * v).sum(-1).view(b, k, 1, 1)

        saliency_map = (weights * activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.interpolate(saliency_map, size=(224, 224), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data

        return saliency_map, logit

def visualize_cam(mask, img):
    """Make heatmap from mask and synthesize GradCAM result image using heatmap and img.
    Args:
        mask (torch.tensor): mask shape of (1, 1, H, W) and each element has value in range [0, 1]
        img (torch.tensor): img shape of (1, 3, H, W) and each pixel value is in range [0, 1]

    Return:
        heatmap (torch.tensor): heatmap img shape of (3, H, W)
        result (torch.tensor): synthesized GradCAM result of same shape with heatmap.
    """
    mask, img = mask.cpu(), img.cpu()
    heatmap = cv2.applyColorMap(np.uint8(255 * mask.squeeze()), cv2.COLORMAP_JET)
    heatmap = torch.from_numpy(heatmap).permute(2, 0, 1).float().div(255)
    b, g, r = heatmap.split(1)
    heatmap = torch.cat([r, g, b])

    result = heatmap + img.cpu()
    result = result.div(result.max()).squeeze()

    return heatmap, result


def find_resnet_layer(arch, target_layer_name):
    """Find resnet layer to calculate GradCAM and GradCAM++

    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'conv1'
            target_layer_name = 'layer1'
            target_layer_name = 'layer1_basicblock0'
            target_layer_name = 'layer1_basicblock0_relu'
            target_layer_name = 'layer1_bottleneck0'
            target_layer_name = 'layer1_bottleneck0_conv1'
            target_layer_name = 'layer1_bottleneck0_downsample'
            target_layer_name = 'layer1_bottleneck0_downsample_0'
            target_layer_name = 'avgpool'
            target_layer_name = 'fc'

    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """
    if 'layer' in target_layer_name:
        hierarchy = target_layer_name.split('_')
        layer_num = int(hierarchy[0].lstrip('layer'))
        if layer_num == 1:
            target_layer = arch.layer1
        elif layer_num == 2:
            target_layer = arch.layer2
        elif layer_num == 3:
            target_layer = arch.layer3
        elif layer_num == 4:
            target_layer = arch.layer4
        else:
            raise ValueError('unknown layer : {}'.format(target_layer_name))

        if len(hierarchy) >= 2:
            bottleneck_num = int(hierarchy[1].lower().lstrip('bottleneck').lstrip('basicblock'))
            target_layer = target_layer[bottleneck_num]

        if len(hierarchy) >= 3:
            target_layer = target_layer._modules[hierarchy[2]]

        if len(hierarchy) == 4:
            target_layer = target_layer._modules[hierarchy[3]]

    else:
        target_layer = arch._modules[target_layer_name]

    return target_layer


def find_densenet_layer(arch, target_layer_name):
    """Find densenet layer to calculate GradCAM and GradCAM++

    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'features'
            target_layer_name = 'features_transition1'
            target_layer_name = 'features_transition1_norm'
            target_layer_name = 'features_denseblock2_denselayer12'
            target_layer_name = 'features_denseblock2_denselayer12_norm1'
            target_layer_name = 'features_denseblock2_denselayer12_norm1'
            target_layer_name = 'classifier'

    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """

    hierarchy = target_layer_name.split('_')
    target_layer = arch._modules[hierarchy[0]]

    if len(hierarchy) >= 2:
        target_layer = target_layer._modules[hierarchy[1]]

    if len(hierarchy) >= 3:
        target_layer = target_layer._modules[hierarchy[2]]

    if len(hierarchy) == 4:
        target_layer = target_layer._modules[hierarchy[3]]

    return target_layer


def find_vgg_layer(arch, target_layer_name):
    """Find vgg layer to calculate GradCAM and GradCAM++

    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'features'
            target_layer_name = 'features_42'
            target_layer_name = 'classifier'
            target_layer_name = 'classifier_0'

    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """
    hierarchy = target_layer_name.split('_')

    if len(hierarchy) >= 1:
        target_layer = arch.features

    if len(hierarchy) == 2:
        target_layer = target_layer[int(hierarchy[1])]

    return target_layer


def find_alexnet_layer(arch, target_layer_name):
    """Find alexnet layer to calculate GradCAM and GradCAM++

    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'features'
            target_layer_name = 'features_0'
            target_layer_name = 'classifier'
            target_layer_name = 'classifier_0'

    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """
    hierarchy = target_layer_name.split('_')

    if len(hierarchy) >= 1:
        target_layer = arch.features

    if len(hierarchy) == 2:
        target_layer = target_layer[int(hierarchy[1])]

    return target_layer


def find_squeezenet_layer(arch, target_layer_name):
    """Find squeezenet layer to calculate GradCAM and GradCAM++

    Args:
        arch: default torchvision densenet models
        target_layer_name (str): the name of layer with its hierarchical information. please refer to usages below.
            target_layer_name = 'features_12'
            target_layer_name = 'features_12_expand3x3'
            target_layer_name = 'features_12_expand3x3_activation'

    Return:
        target_layer: found layer. this layer will be hooked to get forward/backward pass information.
    """
    hierarchy = target_layer_name.split('_')
    target_layer = arch._modules[hierarchy[0]]

    if len(hierarchy) >= 2:
        target_layer = target_layer._modules[hierarchy[1]]

    if len(hierarchy) == 3:
        target_layer = target_layer._modules[hierarchy[2]]

    elif len(hierarchy) == 4:
        target_layer = target_layer._modules[hierarchy[2] + '_' + hierarchy[3]]

    return target_layer


def denormalize(tensor, mean, std):
    if not tensor.ndimension() == 4:
        raise TypeError('tensor should be 4D')

    mean = torch.FloatTensor(mean).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)
    std = torch.FloatTensor(std).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)

    return tensor.mul(std).add(mean)


def normalize(tensor, mean, std):
    if not tensor.ndimension() == 4:
        raise TypeError('tensor should be 4D')

    mean = torch.FloatTensor(mean).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)
    std = torch.FloatTensor(std).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)

    return tensor.sub(mean).div(std)


class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return self.do(tensor)

    def do(self, tensor):
        return normalize(tensor, self.mean, self.std)

    def undo(self, tensor):
        return denormalize(tensor, self.mean, self.std)

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:
#@title Visualizing network activations { display-mode: "form" }

#@markdown - Interpretability is a very important factor in deep learning, especially healthcare with deep learning. 
#@markdown Interpretable deep learning models are more reliable and can sometimes result in new insights in the disease. 
#@markdown - Here we use GradCAM++ to visualize the activations, i.e. which
#@markdown part of the image leads to the predictions and showed the performance in 4 randomly selected images.
#@markdown - As shown in the images, the model identified the region of interest correctly for the first, second and fourth image but 
#@markdown was confused by the artifacts in the third image.

from torchvision.utils import make_grid, save_image

model = EfficientNetwork(output_size=1, b4=False, b2=True).to(device)
# model.load_state_dict(torch.load('working//Epoch15_ValidAcc_0.774_ROC_0.852.pth'))
model.eval()

train_df = pd.read_csv(train_csv)
train_df = train_df.iloc[40:45, :].reset_index(drop=True)

model_dict = dict(type='efficientnet', arch=model, layer_name='swish', input_size=(224, 224))
model_gradcampp = GradCAMpp(model_dict, True)

# Data object and Loader
example_data = MelanomaDataset(train_df, vertical_flip=0.5, horizontal_flip=0.5,
                                is_train=False, is_valid=True, is_test=False)
example_loader = torch.utils.data.DataLoader(example_data, batch_size=4, shuffle=False)

original_data = MelanomaDataset(train_df, vertical_flip=0.5, horizontal_flip=0.5,
                                is_train=False, is_valid=False, is_test=False, is_original=True)
original_loader = torch.utils.data.DataLoader(original_data, batch_size=4, shuffle=False)

for (image, csv_data), labels in example_loader:
    image_example = image
    break

for (image, csv_data), labels in original_loader:
    original_example = image
    break

images = []

for i in range(len(image_example)):
    current_image = image_example[i].view(1,3,224,224)
    current_original = original_example[i].view(1,3,224,224)

    mask, _ = model_gradcampp(current_image.cuda())
    heatmap, result = visualize_cam(mask, current_original)
    images.append(torch.stack([current_original.squeeze().cpu(), heatmap, result], 0))

print(len(images))
images = make_grid(torch.cat(images, 0), nrow=3)
output_dir = 'outputs'
os.makedirs(output_dir, exist_ok=True)
output_name = 'cam.png'
output_path = os.path.join(output_dir, output_name)
save_image(images, output_path)
PIL.Image.open(output_path)



# Discussion on the performance of the model

- The neural network model we trained achieved ~0.85 AUC and which is not too bad. If we use the whole SIIM-ISIC data to train our model we can reach ~0.95 AUC.
- However, no matter how good the model performs on this dataset, the model is not ready to be applied in clinic yet.
- For example, most images in this dataset are from Caucasian patients. Skin cancer in people of color may have very different clinical presentations [(Paper)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2757062/). If we use our current model to screen patients of color, the model is very likely to misdiagnose some cases. 
- One way to solve this problem is to add skin cancer images from people of color to our training dataset. It's important to think through the use cases and potential biases before collecting and constructing your custom datasets.

Congratulations, you have completed case study 3 in the ‘AI for Clinician Champions Certificate' program!  
https://michener.ca/ce_course/ai-for-clinician-champions-certificate-program/

This program is offered by Michener Institute & Vector Institute for clinicians who wish to learn more about AI in Healthcare.

Instructor: Dr. Devin Singh (@DrDevSK) | Assignment Developer: Jianan Chen | Course Tutors: Jianan Chen, Flora Wan, and S. Alex Yun | Course Director: Shingai Manjengwa (@Tjido) 

***Never stop learning!***